## 4.1 compute the parity of a word
### Bitwise operator

Let x = 10 (0000 1010 in binary) and y = 4 (0000 0100 in binary)

Operator| Meaning | Example
|---|---|---|
& | Bitwise AND	| x & y = 0 (0000 0000)
\| |	Bitwise OR	| x | y = 14 (0000 1110)
~	| Bitwise NOT	| ~x = -11 (1111 0101)
^	| Bitwise XOR	| x ^ y = 14 (0000 1110)
\>\>	| Bitwise right shift |	x >> 2 = 2 (0000 0010)
\<\<	| Bitwise left shift	|x << 2 = 40 (0010 1000)

#### NOT
```
NOT 0111  (decimal 7)
  = 1000  (decimal 8)
```
#### AND
```
    0101 (decimal 5)
AND 0011 (decimal 3)
  = 0001 (decimal 1)
```

#### OR

```
   0010 (decimal 2)
OR 1000 (decimal 8)
 = 1010 (decimal 10)
```

#### XOR
```
    0101 (decimal 5)
XOR 0011 (decimal 3)
  = 0110 (decimal 6)
```

In [15]:
# 4.1 brute force
def parity(x):
    result=0
    while x:
        result ^= x & 1 # result = result XOR (x & 1), if x !=0
        x >>= 1 # x=x(binary) right shift 1
    return result

#### eg.
```
x=4 (bi: 100)
1: x=100(bi) ; result = 000 XOR (100 & 001) = 000 XOR 000 = 000
2: x=010(bi) ; result = 000 XOR (010 & 001) = 000 XOR 000 = 000
3: x=001(bi) ; result = 000 XOR (001 & 001) = 000 XOR 001 = 001
4: x=000(bi)
return result=1
```

```
x=5  (bi: 101)
1: x=101(bi) ; result = 000 XOR (101 & 001) = 000 XOR 001 = 001
2: x=010(bi) ; result = 001 XOR (010 & 001) = 001 XOR 000 = 001
3: x=001(bi) ; result = 001 XOR (001 & 001) = 001 XOR 001 = 000
4: x=000(bi)
return result=0
```

In [30]:
# 4.1 erasing the lowest set bit
def parity(x):
    result=0
    while x:
        result ^=  1 
        x &= x-1 # Drops the lowest set bit of x
    return result

#### eg.
```
x=4 (bi: 100)
1: x=100             ; result = 000 XOR 001 = 001
2: x=100 & 011 = 000 ; 
return result=1

x=5  (bi: 101)
1: x=101             ; result = 000 XOR 001 = 001
2: x=101 & 100 = 100 ; result = 001 XOR 001 = 000
3: x=100 & 011 = 000 ; 
4: x=000
return result=0
```

In [32]:
5&4

4